# Notes for Stanford CS143 Compilers

## Week 01: Introduction & the Cool Programming Language

**How to Run the Program**

- Compiler (Offline)
  - $Program \rightarrow Compiler \rightarrow Execute$
  - $Execute + Data \rightarrow Output$
- Interpreters (Online)
  - $Program + Data \rightarrow Interpreter \rightarrow Output$

**Compiler Concept**

- (Syntactic) **Lexical Analysis**
  - Concept: Divide program text into words or tokens.
  - Input: text
  - Output: words or tokens
  - Sample Input: `if x == y then z = 1; else z = 2;`
  - Sample Output: `#IF #ID(x) #EQAUL #ID(y) #THEN ...`
- (Syntactic) **Parsing**
  - Concept: Diagramming Sentences.
  - Input: Tokens
  - Output: Abstruct Semantic Tree
  - Sample Input: #INT(5) #PLUS #INT(3) #MULTIPLY #INT(5)
  - Sample Output: `(#PLUS (#INT(5))  (#MULTIPLY (#INT(3))  (#INT(5))))`
- (Types scope) **Semantic Analysis**
  - Concept: Catch inconsistencies.
  - Sample Input: `{ int Jack=3; { int Jack=4; cout << Jack; } }`
  - Question: What is the value?
- **Optimization**
  - Concept: Run faster/Use less memory/Use low power/network.
- (Translation) **Code Generation**
  - Concept: Produce assembly code.

**Related Questions**

- Why are there so many programming languages?
  - Application domains have distinct/conflicting needs.
- Why are there new programming languages?
  - Programming training is the dominant cost for a programming language.
  - Wild-used languages are **slow to change**.
  - Easy to start a new language: when **productivity** > **training cost**
  - Languages adopted to fill a void. (**Void** means new techniques.)
- What is a good programming languages?
  - There is **no** universally accepted metric for language design.


## Week 02: Lexical Analysis & Finite Automata

### 03 Lexical Analysis

**Token Class (or Class)**

Classify **program substrings** according to **role** (token class), **class** corresponding to **sets of strings**:

- **Identifier** : string of letters or digits, starting with a letter
- **Integer**: a non-empty string of digits
- **Keyword**: else, if, begin, ...
- **Whitespace**: a non-empty sequence of blanks, newlines, tabs

**Goal of Lexical Analysis**

- Definition
	- **lexeme**: A lexeme is a sequence of characters that matches the pattern for a token.
	- **token**: A token is a pair consisting of the token name and the value.
- Concept
	- Parttition the input string into lexemes.
	- Identity the token of each lexeme.
	- Communicate tokens to the parser.
- Input: Program Substrings
- Output: Tokens
- Sample Input: `string (foo=42)`
- Sample Output: `<class, "string">, <'('>, <ID, "foo">, <Operator, "=">, <"Int", "42>, <')'>`
- Remark
	- **Left-to-Right** scan => **lookahead** required.

#### Regular Languages

Regular Expression:

- Concept: Regular expressions (syntax) specify regular languages (set of strings).
- 2 base cases
  - Single Character: $'c' = \{ "c" \}$
  - Empty String: $\epsilon = \{ "" \}$
- 3 compound expressions
  - Union: $A + B = \{a\ |\ a \in A\} \cup \{ b\ |\ b \in B \}$
  - Concatenation: $AB = \{ab\ |\ a \in A \wedge b \in B \}$
  - Iteration: $
A^* = \bigcup_{i \geq 0} A^i,
\begin{cases}
A^i = A...A \\
A^0 = \epsilon
\end{cases}
$

**Def.** The **regular expression** over $\Sigma$ are the smallest set of expressions including:
$$
\begin{align}
  R &= \epsilon \\
    &|\ 'c', c\in\Sigma\\
    &|\ R+R\\
    &|\ RR\\
    &|\ R^*
\end{align}
$$

#### Formal Languages

**Def.** Let $\Sigma$ be a set of characters (an alphabet).
A **language** over $\Sigma$ is a set of strings of characters drawn from $\Sigma$.
Meaning function $L$ maps **regular expressions** to **set of strings**.

$$
\begin{align}
L(\epsilon) &= \{ "" \} \\
L('c') &= \{ "c" \} \\
L(A+B) &= L(A) \cup L(B) \\
L(AB) &= \{ab\ |\ a \in L(A) \wedge b \in L(B)\} \\
L(A^*) &= \bigcup_{i \geq 0} L(A)^i \\
\end{align}
$$

**Q**:Why use a meaning function?

- Make clear what is syntax, what is semantics.
- Allow us to consider notation as a seperate issue.
- Because exp and meanings are not 1-1.
  - Meaning is many to one.

### 04 Lexical Specifications

**Lexemes**

- **Keyword**: "if" / "else" / "then" / ...
  - $ 'if' + 'else' + 'then' + ...$
- **Integer**: a non-empty string of digits
  - $digits: '0' + '1' + '2' + ...$
  - $(digit)(digit)^* = digit^+$
- **Identifier**: strings of letters or digits, starting with a letter
  - $letter = 'a' + 'b' + 'c' + ... = [a-zA-Z]$
  - $(letter)(digit+letter)^*$
- **Whitespace**: a non-empty sequence of blanks, newlines, and tabs
  - $('\;' + '\setminus n' + '\setminus t')^+$

**More regular expressions**

- At least one: $A^+ = AA^*$
- Union: $A|B = A+B$
- Option: $A? = A+\epsilon$
- Range: $'a'+'b'+\dots+'z' = [a-z]$
- Excluded Range: $\widehat{[a-z]} = [\wedge a-z]$

**How do we check $program \in L(R)$**?

1. Write a regular expression for the lexemes of each token class
	- Number $= digit^+$
	- Keyword $= 'if'+'else'+\dots$
	- Identifiew $= letter(letter+digit)^*$
	- OpenPar $='('$
	- ...
2. Construct $R$ to match all lexemes.
	- $R = Number + Keyword + Number + ... = R_1 + R_2 + ...$
3. Let input be $x_1...x_n$. Find the longest length $i$ such that $x_1...x_i \in L(R)$.
4. Remove $x_1...x_i$. Go to step 3.
5. If $x_1...x_i \in L(R_a) \cap L(R_b)$, apply $R_{\min(a,b)}$.


## Week 03: Parsing & Top-Down Parsing

### Parsing

What is **parsing**?

- **Goal**: To derive the parse tree of program in grammar.
- **Input**: Sequence of tokens from lexer
- **Output**: Parse tree of the program.
- Sample
	- Input: `IF ID = ID THEN INT ELSE INT FI`
	- Output: `(IF-THEN-ELSE (= ID ID) (INT) (INT))`

#### Context-Free Grammars

- **Goal**: To separate the token string into many pieces such that every piece is acceptable in grammar.
	- **Context-Free** means every two pieces are not related.
	- Parser must distinguish between **valid** and **invalid** strings of tokens under CFGs.
- What we need
	- A language for **describing valid strings of tokens**.
	- A method for distinguishing valid under the language.
- Note. Programming languages always have recursive structures.
	- An **EXPR** is ... (recursive)
		- if EXPR then EXPR else EXPR fi
	- Context-free grammars are a natural notation for this recursive structure.
- Def. A CFG consists of
	- A set of terminals $T$
	- A set of non-terminals $N$
	- A start symbol $S \in N$
	- A set of productions $X \rightarrow Y_1, \dots, Y_N, X\in N, Y_i \in N\cup T\cup \{ \epsilon\}$
		- Ex: $S\rightarrow (S)$
		- How the production works?
			- Begin with a string with only the start symbol $S$
			- Replace any non-terminal $X$ in the string by the right-hand side of some production $X \rightarrow Y_1\dots Y_n$
			- Repeat (2) until there are no non-terminals
				- $X_1\dots X_i X X_{i+1}\dots X_N \rightarrow X_1\dots X_i Y_1\dots Y_k X_{i+1}\dots X_N,\ if\ X\rightarrow Y_1\dots Y_k$
				- $S \rightarrow \dots \rightarrow \alpha_0 \rightarrow \alpha_1 \rightarrow \dots \rightarrow \alpha_n, if\ \alpha_0 \rightarrow^* \alpha_n (\geq 0\ steps)$
- Def. (Language) Use grammar to define language.
	- Let $G$ be a context-free grammar with start symbol $S$. Then the language $L(G)$ of $G$ is $$\{ a_1\dots a_n\ |\ \forall i\ a_i\in T\wedge S\rightarrow^* a_1\dots a_n \}$$
	- Terminals ought to be tokens of the language.
	- Membership in a language is **yes** or **no**; also need parse tree of the input.
	- Must handle errors gracefully.
	- Need an implementation of CFG’s (e.g., bison)
	- Form of the grammar is important.
		- Many grammars generate the same language.
		- Tools are sensitive to the grammar.

#### Derivations

- **Goal**: To define the process from the start symbol to some valid token string.
	- Def. A **derivation** is a sequence of productions applying on a start symbol $S$ to a sequences without any non-terminal.
	- We can draw a derivation as a tree
		- Start symbol is the tree’s root
		- For a production $X \rightarrow Y_1\dots Y_n$ add children $Y_1\dots Y_n$ to node $X$.
	- Sample
		- $G$: $E\rightarrow E+E\ |\ E*E\ |\ (E)\ |\ id$
		- Target: $id*id+id$
		- Derivation: $E \rightarrow E+E \rightarrow E*E+E \rightarrow id*E+E \rightarrow id*id+E \rightarrow id*id+id$
		- Draw derivation as a parse tree: $(((id) * (id)) + (id))$
		- The example is a **left-most** derivation. At each step, replace the left-most non-terminal.
		- Note that right-most and left-most derivations have the same parse tree.
- A parse tree has
	- Terminals at the leaves
	- Non-terminals at the interior nodes
- An **in-order** traversal of the leaves is the original input.
- We are not just interested in whether $s \in L(G)$.
	- We need a parse tree for $s$.
- The relation between derivation and parse tree are many to one.
	- `((id)+(id))`

#### Ambiguity

- **Goal**: Parse tree is used to describe how we explain the program string. If there are two parse trees for the program, it has two meanings.
- Sample
	- $G$: $E\rightarrow E+E\ |\ E*E\ |\ (E)\ |\ id$
	- Target: $id*id+id$
	- This string has two parse trees
		- Derivation: $E \rightarrow E+E \rightarrow E*E+E \rightarrow id*E+E \rightarrow id*id+E \rightarrow id*id+id$
		- Derivation: $E \rightarrow E*E \rightarrow E*E+E \rightarrow id*E+E \rightarrow id*id+E \rightarrow id*id+id$
- Def. A grammar is **ambiguous**, if it has more than one parse tree for some string $s\in L(G)$.
	- A lot of ways to solve it.
	- Most direct method is to rewrite grammar unambiguously.
	- Like, enforces precedence of * over +.
	- Impossible to convert **automatically** an ambiguous grammar to an unambiguous one.
	- Most tools allow precedence and associativity declarations to disambiguate grammars.

#### Error

Error Handling

- **Goal**
	- To detect non-valid programs
	- To translate the valid ones
	- Sample
		- (Lexer) Errors in Lexical: `...$...`
		- (Parser) Errors in Syntax: `...x *%...`
		- (Type checker) Errors in Semantic: `...int x; y=x(3);...`
		- (Tester/User) Errors in Correctness: program
	
Error Handler

- **Goal**
	- Report errors accurately and clearly
	- Recover from an error quickly
	- Not slow down compilation of valid code
- How to do
	- Panic mode
		- Discard tokens until one with a clear role is found. And continue.
		- Bison: use the special terminal error to describe how much input to skip.
	- Error productions
		- Specify known common mistakes in the grammar, like `5x` and `5*x`...
		- Complicates the grammar.
	- Automatic local or global correction
		- Try token insertions and deletions. (Edit distance)
		- Exhaustive search.

#### Abstract Syntax Trees (AST)

- **Goal**: Represent the parse tree and ignore the details.
- **Form**: `(op (op1) (op2),...,(opn))`
	- `(IF-THEN-ELSE (< (x) (5)) (= (y) (4)) (= (y) (3)))`

#### Top-Down Parsing

- Recursive Descent Parsing
	- The parse tree is constructed
		- From the top
		- From left to right
	- Use DFS to match the token string
		- Let the global `next` point to the next input token: `TOKEN *next = array;`
		- Try to match a given token terminal: `bool term(TOKEN tok) { return *next++ == tok; }`
		- Try to mathc the $n^{th}$ production of S: `bool Sn() { ... }`
		- For production $E\rightarrow T$: `bool E1() { return T(); }`
		- For production $E\rightarrow T+E$: `bool E2() { return T() && term(PLUS) && E(); }`
		- For all productions of E (with backtracking) ```
			bool E() { TOKEN *save = next; return (next = save, E1()) || (next = save, E2()); }```
- Error in RD
	- Scenario: $S \rightarrow Sa$
		- It goes into an infinite loop.
	- RD does not work in **left-recursive grammar**.
		- Def. A **left-recursive grammar** has a non-terminal $S$, $S\rightarrow^+ S\alpha$, for some $\alpha$
		- Rewrite all production rules to use right-recursion.


## Week 04: Bottom-Up Parsing I & II

### Predictive Parser: LL(1)

#### Introduction

- Concept
  - Look at the next ? tokens.
  - No backtracking.
  - Accept LL(K) grammars. (Left-to-Right, Leftmost derivation, k tokens)
  - At each step, only 1 choice.
  - To avoid ambiguousness, need to **left-factor** the grammar.
- Parsing Table: Leftmode Non-terminal x Next Input Token
  - Use stack to record frontier of parse tree
- Differnce from Recursive Descent
  - For the leftmost non-terminal **S**
  - Look at the next input token **a**
  - Choose the production shown at **[S,a]**
- Reject on reaching error state
- Accept on (end of input && empty stack)

#### First Set

**Def**. (First Set)

$$First(X)=\{t\ |\ X\rightarrow^*t\alpha\}\cup\{\epsilon\ |\ X\rightarrow^*\epsilon\}$$

**Algo**

1. $First(X)=\{t\}, if\ t\ is\ a\ terminal.$
2. $\epsilon \in First(X), if
\begin{cases}
X\rightarrow \epsilon \\
X\rightarrow A_1A_2\dots A_n, and\ \epsilon \in First(A_i), \forall 1\leq i\leq n
\end{cases}$
3. $First(\alpha) \subseteq First(X), if\ X\rightarrow A_1A_2\dots A_n\alpha, and\ \epsilon \in First(A_i), \forall 1\leq i\leq n$

#### Follow Set

**Def**. (Follow Set)

$$Follow(X)=\{t\ |\ S\rightarrow^*\beta Xt\delta\}$$

**Algo**

1. $\$ \in Follow(S)$
2. $First(\beta)-\{\epsilon\}\subseteq Follow(X), for\ each\ A\rightarrow\alpha X\beta$
3. $Follow(A) \subseteq Follow(X), for\ each A\rightarrow\alpha X\beta\ where\ \epsilon\in First(\beta)$

#### How to Construct LL(1) Parsing Tables

- Construct a parsing table **T** for CFG **G**
- For each production $A \rightarrow \alpha$ in G do:
  - For each terminal $t \in First(\alpha)$, $T[A, t] = \alpha$
  - If $\epsilon \in First(\alpha),$, for each $t \in Follow(A)$, $T[A,t] = \alpha$
  - If $\epsilon \in First(\alpha)$ and $ \$ \in Follow(A)$, $T[A,\$] = \alpha$
- If any entry is multiply defined, then G is not LL(1).

### Bottom-Up Parsing 

#### Shift-Reduce Parsing

- Concept
  - Don't need **left-factored** grammars.
  - Reduce the string to the start symbol. (Inverting production)
  - A bottom-up parser traces a rightmost derivation in reverse. $$
\begin{align}
& int*int + int \\
& \textbf{int*T} + int \\
& \textbf{T} + int \\
& T + \textbf{T} \\
& T + \textbf{E} \\
& \textbf{E}
\end{align}
$$
  - Thm. 
    - In some step, let string as $\alpha \beta \gamma$.
    - Assume the next reduction is by $X \rightarrow \beta$. ($\alpha \beta \gamma \rightarrow \alpha X \gamma$)
    - Then $\gamma$ is a string which contains only terminals.
  - Split the string as $L_{Str} | R_{Str}$.
  - $L_{Str}$ contains non-terminals and terminals. $R_{Str}$ contains only terminals.
- Actions
  - **Shift**: Move | one place to the right. Shift a terminal to the left string.
  - **Reduce**: Apply an inverse production at the right of $L_{str}$. $$
for\ A \rightarrow xy, Cbxy|ijk \Rightarrow CbA|ijk $$
- Implementation
  - Use a stack to maintain $L_{Str}$.
- If is's legal to shift or reduce, there is a **shift-reduce** conflict.
- If is's legal to reduce by 2 productions, there is a **reduce-reduce** conflict.

#### Term. (Handle)

- Scenario
  - Grammar: $E \rightarrow T+E|E$ & $T \rightarrow int*T|int|(E)$
  - At the steop $int|*int+int$
  - If we reduce by ($T \rightarrow int$) given ($T | *int+int$), it will fail.
- Target: Want to reduce only if the result can still be reduced to the start symbol.
- A **handle** is the rhs of the production that you can trace back in the parse tree.
- Assume a rightmost derivation $$S \rightarrow^* \alpha X \omega \rightarrow \alpha\beta\omega$$
  - Then $\beta$ is a **handle** of $\alpha \beta \omega$.
- Thm.
  - In shift-reduce parsing, handles appear only **at the top of the stack**, never inside.
- Bottom-up parsing algorithms are based on recognizing handles

#### Recognizing Handles

- There are no known efficient algo to recognize handles. But there are good heuristics.
- **Viable Prefix**: $\alpha_1\dots\alpha_n$ is a **viable prefix** if there is an $\omega$ s.t. $\alpha_1\dots\alpha_n\ |\ \omega$ is a state of a shift-reduce parser.
  - A viable prefix is always the prefix of some handle.
  - If we can maintain the stack's contents are viable prefixes all the time, no parsing error will occur.
- Thm. For any grammar, the set of viable prefixes is a **regular language**.
- **Item**: An **item** is a production with '.' somewhere on the rhs.
  - Example. The items for $T \rightarrow (E)$ are
    - $T \rightarrow .(E)$
    - $T \rightarrow (.E)$
    - $T \rightarrow (E.)$
    - $T \rightarrow (E).$
  - Example. The items for ($X \rightarrow \epsilon$) are $X \rightarrow .$
  - Items are often called **LR(0) items**.
- Target: To describe the states of the production rule.
  - If we need to use some production rule $R$ to reduce, the top of the stack will be the left string split by '.' on some item of $R$.
  - Example.
    - Input: $(int)$
    - Grammar: $E \rightarrow T+E\ |\ T$ and $T \rightarrow int*T\ |\ int\ |\ (E)$
    - $(E$ is the left string split by '.' on the item $T \rightarrow (E.)$


#### Recognizing Viable Prefixes

Algo.
1. Add a dummy production $S' \rightarrow S$ to $G$
2. The NFA states are the items of $G$
3. For item $E \rightarrow \alpha .X\beta$ add transition: $(E \rightarrow \alpha . X \beta) \rightarrow^X (E \rightarrow \alpha X.\beta)$
4. For item $E \rightarrow \alpha .X\beta$ and production $X \rightarrow \gamma$, add transition: $(E \rightarrow \alpha . X \beta) \rightarrow^\epsilon (X \rightarrow .\gamma)$
5. Every state is an accepting state
6. Start state is $S' \rightarrow .S$

#### Valid Items

- Concept
  - Item $X \rightarrow \beta .\gamma$ is **valid** for a viable prefix $\alpha\beta$ if $$S' \rightarrow^* \alpha X\omega \rightarrow \alpha\beta\gamma\omega$$ by a right-most derivation.
  - An item $I$ is valid for a viable prefix $\alpha(\alpha_1\dots\alpha_n)$ if the DFA accepts $\alpha_1\dots\alpha_n$ and terminates on some state $s$ containing $I$.
  - The items in $s$ describe what the top of the item stack might be after reading input $\alpha_1\dots\alpha_n$.
  

#### LR(0) Parsing

- Assume
  - Stack contains $\alpha_1\dots\alpha_k$
  - Next input is $t$
  - DFA on input $\alpha_1\dots\alpha_k$ teminates in state $s$
- Action
  - Reduce by $X \rightarrow \beta$ if $s$ contains item $X \rightarrow \beta.$
  - Shift if $s$ contains item $X \rightarrow \beta .t\omega$
- Conflict
  - Reduce/Reduce if any state contains two reduce rule.
  - Shift/Reduce if any state has a reduce item and a shift item
  
#### Simple LR Paring (SLR(1) Parsing)

- Assume
  - Stack contains $\alpha_1\dots\alpha_k$
  - Next input is $t$
  - DFA on input $\alpha_1\dots\alpha_k$ teminates in state $s$
- Action
  - Reduce by $X \rightarrow \beta$ if $s$ contains item $X \rightarrow \beta.$ and $t \in Follow(X)$
  - Shift if $s$ contains item $X \rightarrow \beta .t\omega$
- If there are conflicts under these rules, the grammar is not SLR.
- We can use **precedence declarations** to resolve conflicts. \* is grater than \+.
- LR(1)
  - More powerful than SLR(1)
  - Build by (item x lookahead).
  - SLR(1) only checks whether lookahead is in follow set.

## Week 05: Semantic Analysis and Type Checking

### Semantic Analysis

#### Introduction

- Error detection
  - Lexical Analysis: Detects inputs with **illegal tokens**
  - Parsing: Detects inputs with **ill-formed parse trees**
  - Semantic Analysis: **Catches all remaining errors**
    - Example. (Cool)
      1. All idenfifiers are declared
      2. Types
      3. Inheritance relationships
      4. Classes defined only once
      5. ...

#### Scope

- Matching identifier declarations with uses.
- The scope of an identifier is the portion of a program in which that identifier is accessible
- The same identifier may refer to different things in different parts of the program.
- Most languages have static scope.
- A few languages are dynamically scoped.
  - A dynamically-scoped variable refers to the closest enclosing binding in the execution of the program.
- Cool identifier bindings are introduced by
  - Class declarations (introduce class names)
  - Method definitions (introduce method names)
  - Let expressions (introduce object id’s)
  - Formal parameters (introduce object id’s)
  - Attribute definitions (introduce object id’s)
  - Case expressions (introduce object id’s)
- Not all identifiers follow the most-closely nested rule.
  - Example. (class definitions in Cool)
    - Are globally visible throughout the program.
- Attribute names are global within the class in which they are defined.
- Methods may also be redefined (overridden).

#### Symbol Tables

- Much of semantic analysis can be expressed as a recursive descent of an AST
  - Before: Push the identifiers into symbol tables (stack).
  - Recure: Process the children of AST.
  - After: Pop the identifier up from symbol tables (stack).
  - Use stack because there are identifier definitions with the same name.
- Symbol Table Operations
  - `add_symbol(x)`
  - `find_symbol(x)`
  - `remove_symbol(x)`
  - `enter_scope()`
  - `exit_scopt()`
  - `check_scope(x)`
- Because class can be used before being defined, how to manage class?
  - Solution. 2 Pass
    - Pass 1: Gather all class names
    - Pass 2: Do the checking
  - Actually, semantic analysis requires **multiple** passes.

### Type Checking

#### Types

- Meaning: The notion varies in different languages.
- Consensus
  - A set of values
  - A set of operations on those values
- In assembly language level, there are no types. For `add $r1, $r2, $r3`, just add it bitwisely.
- Same type, but not make sense.
  - Function Pointer (int) + Number (int)
- Type system in a language specifies which operations are valid for which types.
- The goal of **type checking** is to ensure that operations are used only with the correct type.
- Three kinds of languages
  - Statically typed: All or almost all checking of types is done as part of compilation.
  - Dynamically typed: Almost all checking of types is done as part of program execution.
  - Untyped: No type checking.
- Competing view
  - Static Typing
    - Static checking catches many programming errors at compile time.
    - Avoids overhead of runtime type checks.
  - Dynamic Typing
    - Static type systems are restrictive.
    - Rapid prototyping difficult within a static type system.
- A lot of code is written in statically typed languages with an “escape” mechanism. Unsafe casts in C, Java.
- People retrofit static typing to dynamically typed languages for optimization, debugging.
- The compiler infers types for every expression.
- Term.
  - **Type Checking** is the process of verifying fully typed programs.
  - **Type Inference** is the process of filling in missing type information.
  
#### Type Checking

- We have seen two examples of formal notation specifying parts of a compiler.
  - Lexical Analysis: Regular expressions
  - Parsing: Context-free grammars
  - Type checking: Logical rules of inference
- Inference Rules
  - Form: If **hypothesis** is true, then **Conclusion** is true.
  - Type checking: If $E_1$ and $E_2$ have certain types, then $E_3$ has a certain type.
  - Notation
    - And: $\wedge$
    - If-Then: $\Rightarrow$
    - $x$ has type $T$: $x:T$
    - It is provable that...: $\vdash$
- Example.
  - If $e_1$ has type **Int** and $e_2$ has type **Int**, then $e_1+e_2$ has type **Int**.
  - ($e_1$ has type **Int** $\wedge$ $e_2$ has type **Int**) $\Rightarrow$ $e_1+e_2$ has type **Int**.
  - $(e_1:Int \wedge e_2:Int) \Rightarrow e_1+e_2:Int$
  - $Hypothesis_1 \wedge \dots \wedge Hypothesis_n \Rightarrow Conclusion$
- Tradition inference rules are writen by $$\frac{\vdash Hypothesis \dots \vdash Hypothesis}{\vdash Conclusion}$$
- Cool type rules: $\vdash e:T$
- Example.
  - [Int] $$\frac{i:Constant}{\vdash i:Int}$$
  - [Add] $$\frac{\vdash e_1:Int \vdash e_2:Int}{\vdash e_1+e_2:Int}$$
  - [1+2] $$\frac{\frac{1:Constant}{\vdash 1:Int}\frac{2:Constant}{\vdash 2:Int}}{\vdash 1+2:Int}$$
- A type system is **sound** if
  - When $\vdash e:T$, then $e$ evaluates to a value of type $T$.
  - We only want sound rules and the more prescise one. $(\vdash i:Int) > (\vdash i:Object)$
- Types are computed in a bottom-up pass over the AST.

#### Type Environment

- Other rules
  - [False] $\frac{}{\vdash false: Bool}$
  - [String] $\frac{s:StringConstant}{\vdash s:String}$
  - (Ignore **SELF_TYPE**) **new T** produce an object of type **T** 
    - [New] $\frac{}{\vdash new T:T}$
  - [Not] $\frac{\vdash e: Bool}{\vdash !e:Bool}$
  - [Loop] $\frac{\frac{\vdash e_1:Bool}{\vdash e_2:T}}{\vdash while\ e_1 loop\ e_2 pool:Object}$
  - [Var] $\frac{x\ is\ a\ variable}{\vdash x: ?}$
    - We need the type rule.
- **Type Environment**
  - A **type environment** gives types for **free** variables.
    - A type environment is a function from **Object Identifiers** to **Types**
    - A variable is **free** in a expression if it is not defined within the expression.
  - Let $O$ be a function from **Object Identifiers** to **Types**.
  - Writen by: $O\vdash e:T$
  - Add to every rules.
    - [Int] $$\frac{i:Constant}{O\vdash i:Int}$$
    - [Add] $$\frac{O\vdash e_1:Int\ O\vdash e_2:Int}{O\vdash e_1+e_2:Int}$$
    - [Var] $$\frac{O(x)=T}{O\vdash x:T}$$
    - [Let-No-Init] $$\frac{O[T_0/x]\vdash e_1:T_1}{O\vdash let\ x:T_0\ in\ e_1:T_1}$$
    - Remark
      - $O[T/x](x) = T$
      - $O[T/x](y) = O(y)$
  - Remark
    - The type environment gives types to the free identifiers in the current scope.
    - The type environment is passed down the AST from the root towards the leaves.
    - Types are computed up the AST from the leaves towards the root.

#### Subtyping

- Consider **let** rule
  - [Let-Init] $$\frac{O\vdash e_0:T_0\\O[T_0/x]\vdash e_1:T_1}{O\vdash let\ x:T_0\leftarrow e_0\ in\ e_1:T_1}$$
- Define a relation $\leq$ on classes
  - $X\leq X$
  - $X\leq Y$ if $X$ inherits from $Y$
  - $X\leq Z$ if $X\leq Y$ and $Y\leq Z$
    - [Let-Init] $$\frac{O\vdash e_0:T_0\\O[T_0/x]\vdash e_1:T_1\\T_0\leq T}{O\vdash let\ x:T\leftarrow e_0\ in\ e_1:T_1}$$
    - [Assign] $$\frac{O(x)=T_0\\O\vdash e_1:T_1\\T_1\leq T_0}{O\vdash x\leftarrow e_1:T_1}$$
- Let $O_c(x)=T$ for all attributes $x:T$ in class $C$
  - [Attr-Init] $$\frac{O_c(x)=T_0\\O_c\vdash e_1:T_1\\T_1\leq T_0}{O_c\vdash x:T_0\leftarrow e_1}$$
- How to decide the return type of if statement?
  - if $e_0$ then $e_1$ else $e_2$ fi
  - The best way we can do is the smallest **supertype** larger than the type of $e_1$ or $e_2$.
  - $lub(X,Y)$, the least upper bound of $X$ and $Y$, is $Z$ if 
    - $X\leq Z \wedge Y \leq Z$, $Z$ is an upper bound.
    - $X\leq Z' \wedge Y \leq Z' \Rightarrow Z\leq Z'$, $Z$ is an upper bound.
  - In COOL, the least upper bound of two types is their **least common ancestor** in the inheritance tree.
  - [If-Then-Else] $$\frac{O\vdash e_0:Bool\\O\vdash e_1:T_1\\O\vdash e_2:T_2}{O\vdash if\ e_0\ then\ e_1\ else\ e_2\ fi:lub(T_1,T_2)}$$
  - [Case] $$\frac{O\vdash e_0:T_0\\O[T_1/x_1]\vdash e_1:T_{1`}\\\dots\\ O[T_n/x_n]\vdash e_n:T_{n'}}{O\vdash case\ e_0\ of\ x_1:T_1\rightarrow e_1;\dots ;x_n:T_n\rightarrow e_n;esac:lub(T_1,T_2)}$$
  
#### Typing Methods

- In COOL, method and object identifiers live in different namespaces.
  - A method **foo** and an object **foo** can coexist in the same scope.
  - We use a separate mapping $M$ For method signatures $$M(C,f)=(T_1,\dots,T_n,T_{n+1})$$ means in class $C$ there is a method $f$ $$f(x_1:T_1,\dots,x_n:T_n):T_{n+1}$$
  - [Dispatch] $$\frac{O,M\vdash e_0:T_0\\ O,M\vdash e_1:T_1\\ \dots\\ O,M\vdash e_n:T_n\\ M(T_0, f)=(T_{1'},\dots,T_{n'},T_{n+1})\\ T_i\leq T_{i'},\ for\ 1\leq i\leq n}{O,M\vdash e_0.f(e_1,\dots,e_n):T_{n+1}}$$
  - [Static Dispatch] $$\frac{O,M\vdash e_0:T_0\\ O,M\vdash e_1:T_1\\ \dots\\ O,M\vdash e_n:T_n\\ T_0\leq T\\ M(T, f)=(T_{1'},\dots,T_{n'},T_{n+1})\\ T_i\leq T_{i'},\ for\ 1\leq i\leq n}{O,M\vdash e_0@T.f(e_1,\dots,e_n):T_{n+1}}$$
- The **method environment** must be added to all rules.
- In most cases, $M$ is passed down but not actually used. (Only the dispatch rules use $M$)
  - [Add] $$\frac{O,M\vdash e_1:Int\ O,M\vdash e_2:Int}{O,M\vdash e_1+e_2:Int}$$
- To derive the type of **SELF_TYPE**, we need to know the class in which an expression appears.
- The full type environment for COOL
  - A mapping $O$ giving types to object id's
  - A mapping $M$ giving types to methods
  - The current class $C$
  - [Add] $$\frac{O,M,C\vdash e_1:Int\ O,M,C\vdash e_2:Int}{O,M,C\vdash e_1+e_2:Int}$$
- General Themes
  - Type rules are defined on the structure of expressions.
  - Types of variables are modeled by an environment.
  
#### Implementation of Typing Checking

- COOL type checking can be implemented in a single traversal over the AST.
- Type environment is passed down the tree from parent to child.
- Types are passed up the tree from child to parent.
  - [Add] $$\frac{O,M,C\vdash e_1:Int\ O,M,C\vdash e_2:Int}{O,M,C\vdash e_1+e_2:Int}$$
  - ```c
    TypeCheck(Environment, e1+e2) = {
      T1 = TypeCheck(Environment, e1);
      T2 = TypeCheck(Environment, e2);
      Check T1 == T2 == Int;
      return Int;
    }
    ```
  - [Let-Init] $$\frac{O\vdash e_0:T_0\\O[T_0/x]\vdash e_1:T_1\\T_0\leq T}{O\vdash let\ x:T\leftarrow e_0\ in\ e_1:T_1}$$
  - ```c
    TypeCheck(Environment, let x:T <- e0 in e1) = {
      T0 = TypeCheck(Environment, e0);
      T1 = TypeCheck(Environment.add(x:T), e1);
      Check subtype(T0,T);
      return T1;
    }
    ```

## Week 06: Cool Type Checking & Runtime Organization



## Week 07: Code Generation & Operational Semantics



## Week 08: Local Optimization & Global Optimization



## Week 09: Register Allocation & Garbage Collection



## Week 10: Java